# Python中的__init__和__new__
- __init__(self):构造函数
    - 实例化时进行初始化的工作
    - 有特殊的名称的写法
    - 当实例对象创建完成后被调用的，然后设置对象属性的一些初始值
    - 第一个参数必须有，通用self。参数写入放在括号中
- __new__:
    - 实例创建之前就被调用，它的任务就是创建实例然后然后将它传递给init进行初始化，是个静态方法
    - 一般来说自己没有设置__new__，默认调用系统设置的。
    - new函数必须要有返回值
    _ __new__函数的应用：
           1、__new__方法主要是当你继承一些不可变的class时(比如int, str, tuple)， 提供给你一个自定义这些类的实例化过程的途径
           2、还有就是实现自定义的metaclass
           3、用__new__来实现单例
- __new__函数的本质
    - （1）__new__方法是定义在元类type里面的，作用就是专门创建实例的。

    - （2）__new__的本质上是一个“类方法”，故而第一个参数为cls，但是因为系统知道它是类方法，所以有不需要显式添加@classmethod

    - （3）__new__必须具有返回值，否则无法创建对象，因为__init__函数需要这个返回值

    - （4）自己在定义__new__的时候，参数要与__init__函数的参数匹配，我可以不用到这些参数，但一定要匹配。或者可以使用*arg和**args的形式。

In [1]:
class Student():
    def __new__(cls, *args, **kwargs):
        print("我是new函数啊") # 这句话是为了追踪new函数
        print(type(cls))  # 追踪new函数的执行过程
        print(cls)
        return object.__new__(cls) #调用父类的（object）的new方法，返回一个Student实例，这个实例传递给init的self参数
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print("hello，我是init")
        
    def studying(self):
        print("我爱学习，我在学习")
        
if __name__ == "__main__":
    s = Student("liutao", 23)
    #一旦创建实例对象，就会调用new函数和init函数

我是new函数啊
<class 'type'>
<class '__main__.Student'>
hello，我是init


In [2]:
class Parent():
    def __new__(cls, *args, **kwargs):
        print("我是父类的new函数") # 这句话是为了追踪new函数
        print(type(cls))  # 追踪new函数的执行过程
        print(cls)
        return object.__new__(cls) #调用父类的（object）的new方法，返回一个Student实例，这个实例传递给init的self参数
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print("hello，我是父类的init")
        
    def studying(self):
        print("我爱学习，我在学习")
        

class Child(Parent):
    def __new__(cls, *args, **kwargs):
        print("我是子类的new函数") # 这句话是为了追踪new函数
        print(type(cls))  # 追踪new函数的执行过程
        print(cls)
        return Parent.__new__(cls)
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print("我是子类的init")
    
        
if __name__ == "__main__":
    p = Parent("baba", 50)
    print(p.name)
    print(p.age)
    print("===========")
    s = Child("son", 23)
    print(s.name)
    print(s.age)
    #一旦创建实例对象，就会调用new函数和init函数

我是父类的new函数
<class 'type'>
<class '__main__.Parent'>
hello，我是父类的init
baba
50
我是子类的new函数
<class 'type'>
<class '__main__.Child'>
我是父类的new函数
<class 'type'>
<class '__main__.Child'>
我是子类的init
son
23


In [ ]:
class People():
    name = None
    age = 21
    def __init__(self):
        print('Hello!World.')
per = People()

# Python中的__call__函数
- 可以将它们作为输入传递到其他的函数/方法中并调用他们，正如我们调用一个正常的函数那样，类中__call__()函数的意义正在于此。
- 其和_init_()方法区别
        1. __init__()的作用是初始化某个类的一个实例。 
        2. __call__()的作用是使实例能够像函数一样被调用，同时不影响实例本身的生命周期（__call__()不影响一个实例的构造和析构）。但是__call__()可以用来改变实例的内部成员的值。
- _call_方法的存在，让实例可以让类似函数方法那样被直接使用，但是有个前提就是必须这个类是实例化了的，在Python实际使用中必须先实例化对象

In [5]:
class X(object):
    def __init__(self, a, b, ranging):
        self.a = a
        self.b = b
        self.range = ranging
    def __call__(self, a, b):
        self.a = a
        self.b = b
        print('__call__ with （{}, {}）'.format(self.a, self.b))
    def __del__(self, a, b, ranging):
        del self.a
        del self.b
        del self.ranging
#实例化这个类
x_instance = X(1, 2, 3)
#通过_call_()来修改类属性值
x_instance(1,2)
 
# 得到结果：__call__ with （1, 5）

__call__ with （1, 2）


Exception ignored in: <function X.__del__ at 0x0000029550D72AE8>
TypeError: __del__() missing 3 required positional arguments: 'a', 'b', and 'range'


# python中的__del__(析构函数)和引用计数
- 1.析构函数即__del__方法，当使用del 删除对象时，会调用他本身的析构函数，或者当对象在某个作用域中调用完毕，在跳出其作用域的同时析构函数也会被调用（即这个对象在内存中即将被消灭的时候），这样可以用来释放内存空间。　　
　　- __del__()也是可选的，如果不提供，则Python 会在后台提供默认析构函数
　　- 如果要显式的调用析构函数，可以使用del关键字： del obj
- 2.引用计数：pytho中的对象是使用引用计数的方式实现的，也即如果没有任何对象引用到一块内存，那么python就会把这个内存回收
    - 可以使用sys.getrefcount(object)来获取一个对象的引用计数
    - 当我们用del删除一个对象时，是否删除该对象的内存空间取决于引用计数，只有当一个内存对象的引用计数降为0，即没有被变量引用时，解释器的垃圾回收机制才会回收这块内存。只有当内存被真正回收时，__del__方法才会被调用，即才会真正删除内存空间

In [8]:
# 析构函数举例
class Teacher():
    def __init__(self):
        self.__age = "苍老师"
        self.name = "23"
        print("我是构造函数")

    def __del__(self):
        print("我是析构函数")

    def greet(self):
        print("哈喽，my name is liutao")
teacher = Teacher()
teacher.greet()  # 当代码运行完毕，那么就会清理内存 ，被创建的对象就会销毁
#  所以就会调用__del__方法
del teacher  # 删除对象的时候也会进行调用

我是构造函数
我是析构函数
哈喽，my name is liutao
我是析构函数


In [10]:
import sys

class Teacher():
    def __init__(self):
        self.__age = "苍老师"
        self.name = "23"
        print("我是构造函数")

    def __del__(self):
        print("我是析构函数")

    def greet(self):
        print("哈喽，my name is liutao")
teacher = Teacher()
teacher.greet()
ano_teacher = teacher
print(sys.getrefcount(teacher))
#这里的打印结果是3，因为除了teacher和ano_teacher指向它，在调用sys.getrefcount会创建一个临时变量指向它
# 请注意：临时变量在这句语句执行完毕后就会被销毁
del teacher
print(sys.getrefcount(ano_teacher))
print("="*20)
del ano_teacher  #引用计数变为0，就会调用析构函数


我是构造函数
哈喽，my name is liutao
3
2
我是析构函数
